In [1]:
import numpy as np
import pandas as pd
import json
import sqlite3

In [2]:
classifications = pd.read_csv("periodicity-classifications.csv")
classifications['metadata_json'] = [json.loads(q) for q in classifications.metadata]
classifications['annotations_json'] = [json.loads(q) for q in classifications.annotations]
classifications['subject_data_json'] = [json.loads(q) for q in classifications.subject_data]

conn = sqlite3.connect('features.db')
cur = conn.cursor()

In [3]:
iclass = classifications[classifications.workflow_version == classifications['workflow_version'].max()]

clist=[]

In [9]:
clist.clear()
for index, c in iclass.iterrows():
    filename = str(c.subject_data_json[str(c.subject_ids)]['Filename'])
    oid = filename.split("_")[-1]
    oid = int(oid.split(".")[0])
    clist.append({'classification_id':c.classification_id, 'user_name':c.user_name, 'user_id':c.user_id, 
                  'created_at':c.created_at, 'subject_ids':c.subject_ids, 'val':c.annotations_json[0]['value'], 
                  'oid':oid, 'workflow_id':c.workflow_id})
       
##save function

In [10]:
cur.execute("""select classification_id from classifications""")
class_id = cur.fetchall()

for clist_dict in clist:
    if clist_dict['classification_id'] not in class_id:
        string = """insert into classifications values({}, {}, '{}', {}, {}, '{}', '{}', {})""".format(
                clist_dict['classification_id'], clist_dict['oid'], clist_dict['created_at'], clist_dict['subject_ids'], 
                clist_dict['user_id'], clist_dict['user_name'], clist_dict['val'], clist_dict['workflow_id'])    
        cur.execute(string)

In [11]:
cur.execute("""select * from classifications""")
a = cur.fetchall()

In [8]:
cur.execute("""PRaGMA table_info(classifications)""")
cur.fetchall()

[(0, 'classification_id', 'int', 0, None, 1),
 (1, 'oid', 'Bigint', 0, None, 0),
 (2, 'created', 'String', 0, None, 0),
 (3, 'subject_ids', 'int', 0, None, 0),
 (4, 'user_id', 'int', 0, None, 0),
 (5, 'username', 'String', 0, None, 0),
 (6, 'val', 'String', 0, None, 0),
 (7, 'workflow_id', 'int', 0, None, 0)]

In [7]:
cur.execute("""create table classifications(
        classification_id int Primary key,
        oid Bigint,
        created String,
        subject_ids int,
        user_id int, 
        username String,
        val String,
        workflow_id int
        )""")

In [25]:
cur.execute("""select oid, case val when 'Periodic' then '1' else '0' end overall from classifications;""")
b = cur.fetchall()

In [36]:
len(b)

205

In [37]:
cur.execute("""SELECT c1.oid FROM classifications c1 INNER JOIN classifications c2 ON c2.oid = c1.oid AND c1.classification_id <> c2.classification_id""")
cur.fetchall()

[(33772070008511,),
 (32752090003883,),
 (34782080007596,),
 (32752090004203,),
 (1000132040006341,),
 (1000262000005422,),
 (35782020003582,),
 (34782080011018,),
 (1000132080006449,),
 (33772070002579,),
 (1000262000004818,),
 (34782080010649,),
 (35782020006076,),
 (1000102000002821,),
 (1000132050001344,),
 (1010032020000788,),
 (1000202110007561,),
 (34782080002185,),
 (1000372000002541,),
 (1010032010002194,),
 (1000132050008085,),
 (1000202110008098,),
 (1000132050007533,),
 (51892020004020,),
 (1000372000007130,),
 (1000132050004480,),
 (44972080011009,),
 (1010032070004151,),
 (1000132100002568,),
 (1000202110008105,),
 (1000132100006322,),
 (35782020008131,),
 (33762090005135,),
 (1000372000003143,),
 (1000032015000199,),
 (33772070008461,),
 (33772070008461,),
 (1000372000003818,),
 (1000202110002035,),
 (1000262000008772,),
 (1000262000008772,),
 (1010032020000453,),
 (45882050007665,),
 (1000372000001675,),
 (35782020004849,),
 (1010032020002182,),
 (1000132050008290,),
 (

In [23]:
a[0][6]

'Periodic'

In [21]:
a[0]

(99456444,
 33772070008511,
 '2018-03-29 19:59:07 UTC',
 20684033,
 562290,
 'aarozoon',
 'Periodic',
 4500)